# main

In [5]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from scripts.utils import round_float_cols, unique_ordered
from config import paths


if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        print(dept_dir)
        annexe_dept_dir = annexe_dir / dept_dir.name
        annexe_dept_dir.mkdir(exist_ok=True, parents=True)
        # LOAD TABLES
        td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
        td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
        td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
        td006 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)

        break
        # ENVELOPPE PROCESSING


D:\data\dpe_full\depts\1


In [27]:
from scripts.td001_processing import postprocessing_td001
td001 = postprocessing_td001(td001)
td001 = td001.rename(columns ={'id':'td001_dpe_id'})

# ecs

In [95]:
mixte = ('mixte', 'combine', 'chauffage + ecs','chauffage et ecs','lie ','combine ',"idem")


gen_ecs_normalized_lib_matching_dict_ft = {
    "ECS thermodynamique electrique(PAC ou ballon)": [
        ('pompe a chaleur', 'pac', 'thermodynamique', 'air extrait', 'air exterieur', 'air ambiant')],
    
    "ballon a accumulation electrique": [('ballon', 'classique', 'accumulation'), ('electricite', 'electrique','elec.')],
    "ecs electrique indeterminee": [('electricite', 'electrique','elec.')],
    "ecs instantanee electrique": ['instantanee', ('electricite', 'electrique','elec.')],

    'chaudiere mixte gaz': ["chaudiere", mixte, "gaz"],
    'chaudiere mixte fioul': ["chaudiere", mixte, "fioul"],
    'chaudiere mixte bois': [("bois", "biomasse")],

    'chauffe-eau gaz independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"),
                                      "fioul"],
    "ecs collective reseau chaleur": ["reseau", "chaleur"],

    'chaudiere gaz': ["chaudiere", "gaz"],
    'chaudiere gpl': ["chaudiere", "gpl"],

    'chaudiere fioul': ["chaudiere", "fioul"],
    'chaudiere non déterminee': ["chaudiere"],
    'production mixte indeterminee': [ mixte],

}

In [100]:


def postprocessing_ecs_ft(td005):
    td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
    vr_ecs = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
    sys_ecs_lib_infer_ft=vr_ecs.apply(lambda x:affect_lib_by_matching_score(x,gen_ecs_normalized_lib_matching_dict_ft))

In [101]:
from scripts.utils import strip_accents,affect_lib_by_matching_score

In [102]:
td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
vr_ecs = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))

unique_gen_ecs = vr_ecs.unique()
gen_ecs_lib_infer_dict = {k: affect_lib_by_matching_score(k, gen_ecs_normalized_lib_matching_dict_ft) for k in
                         unique_gen_ecs}
sys_ecs_lib_infer_ft=vr_ecs.replace(gen_ecs_lib_infer_dict)

df=pd.concat([vr_ecs.to_frame('vr'),sys_ecs_lib_infer_ft.to_frame('lib'),td005_ecs.td001_dpe_id],axis=1)

df_ecs =df.merge(td001[['td001_dpe_id','nom_methode_dpe_norm']],on='td001_dpe_id',how='right')

df_ecs.loc[df_ecs.nom_methode_dpe_norm=='FACTURE'].lib.fillna('NA').value_counts()

ecs electrique indeterminee                      10790
ballon a accumulation electrique                  8548
indetermine                                       5413
chaudiere mixte gaz                               4963
ECS thermodynamique electrique(PAC ou ballon)     3782
chaudiere gaz                                     3511
production mixte indeterminee                     3482
chaudiere mixte fioul                             2355
NA                                                1261
chaudiere non déterminee                           779
chaudiere fioul                                    571
ecs collective reseau chaleur                      513
chaudiere gpl                                      308
chaudiere mixte bois                               196
chauffe-eau gaz independant                        193
ecs instantanee electrique                         130
chauffe-eau gpl independant                         13
chauffe-eau fioul independant                        3
Name: lib,

In [103]:
sel=df_ecs.loc[df_ecs.lib=='indetermine']
vc = sel.vr.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

In [94]:
sel.vr.loc[sel.vr.str.contains('pac')]

1487      - pac air/eau a accumulation vertical 190 litres
1761                                       pac unical 2009
3730      - preparateur thermodynamique, appoint elec. ...
4175             - magna aqua 300/2, appoint elec. pac ecs
4717        - odyssee 2 / aeromax 3, appoint elec. pac ecs
                               ...                        
92027     - alfea ex   alfea extensa duo a i  6, appoin...
92036     - alfea ex   alfea extensa duo   8, appoint e...
92040                           pac alfea extensa duo ai 6
92136     - alfea ex   alfea extensa duo   6, appoint e...
92181                           pac alfea extensa duo ai 5
Name: vr, Length: 1821, dtype: object

In [104]:
vc.sort_values(ascending=False).iloc[0:40].to_dict()

{'': 4877.0,
 '-': 2918.0,
 'production': 1645.0,
 ':': 1285.0,
 'collective<br/>': 908.0,
 'ballon': 825.0,
 'chaude': 747.0,
 '25': 681.0,
 'litres': 581.0,
 'condens': 567.0,
 'volume': 557.0,
 "d'eau": 529.0,
 'en': 514.0,
 'c': 500.0,
 'green': 479.0,
 'individuelle': 460.0,
 'habitable': 457.0,
 'chauffe<br/>': 457.0,
 "d'ecs": 429.0,
 'installation': 416.0,
 'mira': 386.0,
 'themaplus': 311.0,
 'stockage': 309.0,
 'a': 306.0,
 'duo': 305.0,
 'atlantic': 297.0,
 'ff': 293.0,
 'de': 277.0,
 'eu': 248.0,
 '1<br/>': 235.0,
 'eau': 231.0,
 'gaz': 231.0,
 'f': 225.0,
 'naia': 199.0,
 'source': 195.0,
 'ecs': 194.0,
 'varmax': 185.0,
 '1000': 184.0,
 '30': 184.0,
 'evo': 178.0}

# chauffage

In [79]:
# GEN CH NORMALIZED DICT
gen_ch_normalized_lib_matching_dict = {"pac air/air": ['pac', 'air/air', ('electricite', 'electrique')],
                                       "pac air/eau": ['pac', 'air/eau', ('electricite', 'electrique')],
                                       "pac eau/eau": ['pac', 'eau/eau', ('electricite', 'electrique')],
                                       "pac geothermique": ['pac', (
                                           'geothermique', 'géothermique', 'géothermie', 'geothermie'),
                                                            ('electricite', 'electrique')],
                                       'panneaux rayonnants electriques nfc': ['panneau', ('electricite', 'electrique'),
                                                                               'nfc'],
                                       'radiateurs electriques': ['radiateur', ('electricite', 'electrique')],
                                       'plafonds/planchers rayonnants electriques nfc': [('plancher', 'plafond'),
                                                                                         ('electricite', 'electrique')],
                                       "convecteurs electriques nfc": ['convecteur', ('electricite', 'electrique'),
                                                                       'nfc'],
                                       "poele ou insert bois": [('poele', 'insert'), ('bois', 'biomasse')],
                                       "poele ou insert fioul/gpl": [('poele', 'insert'), ('fioul', 'gpl')],

                                       "autres emetteurs a effet joule": [('electricite', 'electrique')],
                                       "reseau de chaleur": ['reseau', 'chaleur'],
                                       "convecteurs bi-jonction": ['bi', 'jonction', ('electricite', 'electrique')],
                                       }

for type_chaudiere, type_chaudiere_keys in zip(['standard', 'basse temperature', 'condensation', 'non déterminee'],
                                               [('standard', 'classique'), 'basse temperature',
                                                ('condensation', 'condenseurs'), None]):
    for energie in ['fioul', 'gaz']:
        if type_chaudiere_keys is not None:
            gen_ch_normalized_lib_matching_dict[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie,
                                                                                            type_chaudiere_keys]
        else:
            gen_ch_normalized_lib_matching_dict[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie
                                                                                            ]
    energie_gaz = gen_ch_normalized_lib_matching_dict[f'chaudiere gaz {type_chaudiere}'][1]
    energie_gaz = (energie_gaz, 'gpl', 'butane', 'propane')
    gen_ch_normalized_lib_matching_dict[f'chaudiere gaz {type_chaudiere}'][1] = energie_gaz

gen_ch_normalized_lib_matching_dict['chaudiere bois/biomasse'] = ['chaudiere',
                                                                  ('bois', 'biomasse')]
gen_ch_normalized_lib_matching_dict['chaudiere electrique'] = ['chaudiere',
                                                               ('electricite', 'electrique')]

In [80]:
td005_ch = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '16']
vr_ch = td005_ch.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))

unique_gen_ch = vr_ch.unique()
gen_ch_lib_infer_dict = {k: affect_lib_by_matching_score(k, gen_ch_normalized_lib_matching_dict) for k in
                         unique_gen_ch}
sys_ch_lib_infer_ft=vr_ch.replace(gen_ch_lib_infer_dict)

df=pd.concat([vr_ch.to_frame('vr'),sys_ch_lib_infer_ft.to_frame('lib'),td005_ch.td001_dpe_id],axis=1)

df_ch =df.merge(td001[['td001_dpe_id','nom_methode_dpe_norm']],on='td001_dpe_id',how='right')

df_ch.loc[df_ch.nom_methode_dpe_norm=='FACTURE'].lib.fillna('NA').value_counts()

indetermine                                      8978
chaudiere gaz non déterminee                     6900
chaudiere gaz condensation                       4997
chaudiere fioul non déterminee                   4208
radiateurs electriques                           3484
poele ou insert bois                             3482
panneaux rayonnants electriques nfc              3180
convecteurs electriques nfc                      2871
autres emetteurs a effet joule                   2384
chaudiere gaz standard                           2043
chaudiere gaz basse temperature                  1991
plafonds/planchers rayonnants electriques nfc    1079
NA                                               1020
chaudiere bois/biomasse                           978
chaudiere fioul condensation                      827
chaudiere fioul standard                          770
reseau de chaleur                                 737
pac air/air                                       489
chaudiere fioul basse temper

In [ ]:
from collections import Counter
result = Counter(" ".join(df['text'].values.tolist()).split(" ")).items()
result